<a href="https://colab.research.google.com/github/manpazito/311-neighborhood-equity/blob/dev/data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Analysis and GIS Storytelling

In this notebook we will explore 311 + ACS Data.

_Note: This is a starter / placeholder file for Felix / Leslie to work on._


## Loading Data from Google Drive

Don't edit this. This is just for loading in all the data.


In [1]:
# Initialize
!git clone -q https://github.com/manpazito/311-neighborhood-equity.git
!pip install -q -r 311-neighborhood-equity/requirements.txt
!mkdir -p data
!mkdir -p visualizations
!cp -r 311-neighborhood-equity/data/* data/
!cp -r 311-neighborhood-equity/figures/* visualizations/
!rm -rf 311-neighborhood-equity
!rm -rf sample_data
# Note: Make sure to refresh data in the `Files` tab!

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.0/642.0 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.2/351.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 19.7 MB/s eta 0:00:00


# Creating Visualizations


In [ ]:
#Importing relevant packages

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import os
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

In [ ]:
# Load 311 service requests and ACS data (post-processed)
serv_req_gdf = gpd.read_parquet("data/processed/serv_req_cleaned.parquet")
sf_tracts_acs = gpd.read_file("data/processed/sf_tracts_cleaned_2023.gpkg")

print("311 Service Requests GeoDataFrame:")
display(serv_req_gdf.head())
print("San Francisco Tracts ACS GeoDataFrame:")
display(sf_tracts_acs.head())

In [ ]:
# Create a combined Gdf with both geometries to tie individual requests into Census Tracts

sf_tracts_acs = sf_tracts_acs.to_crs(4326)
joined_Gdf = gpd.sjoin(serv_req_gdf, sf_tracts_acs, how="left", predicate="within")
joined_Gdf.head()

#Examination of Census Tracts

Interactive Map to Examine Broader Demographic Trends and Top Request Categories Aross Tracts


In [ ]:
# Creating an interactive map to look at median household income across census tracts

tooltip_acs = [
    "median_household_income",
    "poverty_rate",
    "total_population",
    "median_home_value",
]

acs_discovery_map = sf_tracts_acs.explore(
    column="median_household_income",
    cmap="RdPu",
    tooltip=tooltip_acs,
    legend=True,
    legend_kwds={"caption": "Median Household Income ($)"},
)

title_discovery_map = """
     <h3 align="center" style="font-size:20px"><b>Median Household Income by Census Tract, San Francisco 2023</b></h3>
"""

acs_discovery_map.get_root().html.add_child(folium.Element(title_discovery_map))
acs_discovery_map.save("visualizations/acs_discovery_map.html")
acs_discovery_map

# Types of Requests by Census Tract and Income


Figure 1: street + sidewalk cleaning requests that are opened and closed in 2023 timescale (2023) per capita per tract and associated graphs (such as relationship between requests per capita and income per tract)

Figure 2: Most popular type of request by tract map + stacked bar perhaps?


In [ ]:
# Find the top categories of requests to consolidate smaller categories as "Other"

req_type_counts = joined_Gdf["category"].value_counts()
top_categories = req_type_counts.index[0:5].tolist()
top_categories

In [ ]:
# Recode to just top categories
joined_Gdf["top_categories"] = joined_Gdf["category"].where(
    joined_Gdf["category"].isin(top_categories), other="Other"
)

In [ ]:
requests_counts = (
    joined_Gdf.groupby(["TRACTCE", "top_categories"]).size().unstack(fill_value=0)
)
# Add group by month for some type of time dimension
tract_requests_total = requests_counts.sum(axis=1)

requests_counts_pct = 100 * requests_counts.div(tract_requests_total, axis=0)
top_category = requests_counts.idxmax(axis=1)
max_category = requests_counts_pct.max(axis=1)

requests_Gdf = sf_tracts_acs
requests_Gdf = requests_Gdf.merge(
    top_category.rename("top request category"), left_on="TRACTCE", right_on="TRACTCE"
)
requests_Gdf = requests_Gdf.merge(
    requests_counts, left_on="TRACTCE", right_on="TRACTCE"
)
requests_Gdf = requests_Gdf.merge(max_category.rename("max percentage"), left_on="TRACTCE", right_on="TRACTCE")
requests_Gdf.head()

In [ ]:
# Trying to just the fill opacity by intensity of top category
requests_Gdf["top_cat_intensity"] = (
    requests_Gdf["max percentage"] /
    requests_Gdf["max percentage"].max()
)

requests_Gdf["top_cat_intensity"] = (
    requests_Gdf["top_cat_intensity"].clip(0.2, 1.0)
)


In [ ]:
# Creating the visualization of top request category by census tract

tooltip_cols2 = ["TRACTCE", "median_household_income", "poverty_rate"] + list(
    requests_counts.columns
)
tooltip_aliases_source = ["Tract ID:" , "Median Household Income:", "Poverty Rate:", "Encampments:", "General Request: Public Works:", "Graffiti:", "Other:", "Parking Enforcement:", "Street And Sidewalk Cleaning:"]

request_source_map = requests_Gdf.explore(
    column="top request category",
    cmap="rainbow",
    tooltip=tooltip_cols2,
      tooltip_kwds= {
        "aliases": tooltip_aliases_source,
        "localize": True,
        "sticky": True,
    },
    legend=True,
     #"top_cat_intensity",
    style_kwds={
        "color": "white",
        "weight": 2,
        "fillOpacity": 0.65
    },
    legend_kwds={"caption": "Request Category"}
)

title_request_source_map = """
     <h3 align="center" style="font-size:20px"><b>Top 311 Request Types by Census Tract, San Francisco 2023</b></h3>
"""

request_source_map.get_root().html.add_child(folium.Element(title_request_source_map))
request_source_map.save("visualizations/request_source_map.html")
request_source_map

In [ ]:
#Creating quintiles based on median household income for analysis

requests_Gdf["income_quintile"] = pd.qcut(
    requests_Gdf["median_household_income"],
    q=5,
    labels=[
        "Q1 (Lowest 20%)",
        "Q2",
        "Q3",
        "Q4",
        "Q5 (Highest 20%)",
    ],
)


In [ ]:
# Creating additional visualization of what income quintile census tracts are in

def truncate_colormap(cmap, minval=0.35, maxval=1.0, n=256):
    return colors.LinearSegmentedColormap.from_list(
        f"trunc({cmap.name},{minval},{maxval})",
        cmap(np.linspace(minval, maxval, n)),
    )

inc_quint_tooltip = {
    "TRACTCE": "Tract ID",
    "median_household_income": "Median Household Income",
    "income_quintile": "Median Household Income Quintile",
}

field_list = ["TRACTCE", "median_household_income", "income_quintile"]
tooltip_aliases = ["Tract ID:", "Median Household Income:", "Poverty Rate:"]

income_quintile_map = requests_Gdf.explore(
    column="income_quintile",
    cmap= "YlGn",
    tooltip = field_list,
    tooltip_kwds= {
        "aliases": tooltip_aliases,
        "localize": True,
        "sticky": True,
    },
    legend=True,
    style_kwds={
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.8
    },
    legend_kwds={"caption": "Median Household Income Quntile"}
)

title_request_source_map = """
     <h3 align="center" style="font-size:20px"><b>Census Tract Median Household Income Quintile, San Francisco 2023</b></h3>
"""

income_quintile_map.get_root().html.add_child(folium.Element(title_request_source_map))
income_quintile_map.save("visualizations/median_household_quintile.html")
income_quintile_map

In [ ]:
# Grouping and finding percentage splits of requests by income group

inc_grouped = requests_Gdf.groupby("income_quintile")[list(requests_counts.columns)].sum()
inc_grouped_pct = inc_grouped.div(inc_grouped.sum(axis=1), axis=0)

In [ ]:
#Creating visual of the breakdown of request types by census tract by median household income quintile

fig, ax = plt.subplots(figsize=(10, 6))

inc_grouped_pct.plot(kind="bar", stacked=True, ax=ax)

category = "Street And Sidewalk Cleaning"

for i, quintile in enumerate(inc_grouped_pct.index):
    value = inc_grouped_pct.loc[quintile, category]

    if value > 0.05:  # only label if segment is big enough
        ax.text(
            i,
            inc_grouped_pct.loc[quintile].loc[:category].sum() - value / 2,
            f"{value:.0%}",
            ha="center",
            va="center",
            fontsize=10,
            color="white",
        )

plt.ylabel("Percentage of Requests")
plt.xlabel("Census Tract Median Household Income Quintile")
plt.title("311 Request Type Percentages by Median Household Income Quntile")
plt.legend(title="Request Type", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.savefig("visualizations/request_type_stacked_bar.jpeg")
plt.show()

# Sidewalk and Street Cleaning Requests per Capita by Census Tract


Scatterplot of Median HH income and per capita requests

Median response time by census tract income


In [ ]:
# Category_requests is a function that finds the number of 311 requests made in a given category per 100 residents by census tract median household income quintile

def category_requests(category_name):
  category_Gdf = joined_Gdf[joined_Gdf["category"] == category_name]
  category_Gdf = category_Gdf[category_Gdf["status"] == "Closed"]

  annual_tract_counts = (
    category_Gdf.groupby(["TRACTCE"]).size().reset_index(name="request_count")
  )
  annual_tract_counts = annual_tract_counts.merge(
    sf_tracts_acs[
        [
            "TRACTCE",
            "total_population",
            "median_household_income",
            "poverty_rate",
            "geometry",
            "share_white",
            "share_black",
            "share_hispanic",
        ]
    ],
    on="TRACTCE",
    how="left",
  )
  annual_tract_counts = annual_tract_counts[annual_tract_counts["total_population"] > 0]
  annual_tract_counts["req_per_capita"] = (
    100 * annual_tract_counts["request_count"] / annual_tract_counts["total_population"]
  )
  annual_counts_Gdf = gpd.GeoDataFrame(
      annual_tract_counts, geometry="geometry", crs=sf_tracts_acs.crs
  )
  annual_counts_Gdf = annual_counts_Gdf[annual_counts_Gdf["total_population"] > 1000]
  annual_counts_Gdf["income_quintile"] = pd.qcut(
    annual_counts_Gdf["median_household_income"],
    q=5,
    labels=[
        "Q1 (Lowest 20%)",
        "Q2",
        "Q3",
        "Q4",
        "Q5 (Highest 20%)",
    ],
  )
  pc_req_grouped = annual_counts_Gdf.groupby("income_quintile")["req_per_capita"].mean()
  return pc_req_grouped

In [ ]:
# Function to make bar charts to show the requests per 100 residents for a given category's quintiles

def make_bar(category_quartiles, category_name: str):
  fig, ax = plt.subplots(figsize=(10, 6))
  category_quartiles.plot(kind="bar", stacked=True, ax=ax)
  norm = colors.Normalize(vmin=category_quartiles.min(), vmax=category_quartiles.max())
  cmap =  truncate_colormap(cm.Blues, 0.4, 1.0)
  for bar in ax.containers[0]:
      height = bar.get_height()
      bar.set_color(cmap(norm(height)))

  for container in ax.containers:
    ax.bar_label(
        container,
        fmt="%.2f",
        label_type="edge",
        padding=3,
        fontsize=10,
    )
  sm = cm.ScalarMappable(norm=norm, cmap=cmap)
  sm.set_array([])
  cbar = plt.colorbar(sm, ax=ax)
  cbar.set_label("Requests per 100 residents")
  plt.title(category_name + " 311 Requests by Median Household Income Quintile")
  plt.xlabel("Census Tract Median Household Income Quintile")
  plt.ylabel(category_name + " 311 Requests per 100 residents")
  plt.savefig("visualizations/"+category_name+"_bar.jpeg")
  plt.show()


In [ ]:
#joined_Gdf["category"].unique()

In [ ]:
#Running to see encampment requests by income quintiles

encampment_quintiles = category_requests("Encampments")
make_bar(encampment_quintiles, "Encampment")

In [ ]:
#Running to see street and sidewalk cleaning requests by income quintiles

streetsw_quintiles = category_requests("Street And Sidewalk Cleaning")
make_bar(streetsw_quintiles, "Street And Sidewalk Cleaning")

In [ ]:
#Running to see Tree maintenance requests by income quintiles

tree_quintiles = category_requests("Tree Maintenance")
make_bar(tree_quintiles, "Tree Maintenance")

In [ ]:
#Running to see Illegal Posting maintenance requests by income quintiles

postings_quintiles = category_requests("Illegal Postings")
make_bar(postings_quintiles, "Illegal Postings")

In [ ]:
#Running to see Sewer Issues maintenance requests by income quintiles

postings_quintiles = category_requests("Sewer Issues")
make_bar(postings_quintiles, "Sewer Issues")